In [1]:
versioninfo()

Julia Version 1.3.1
Commit 2d5741174c* (2019-12-30 21:36 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: Intel(R) Xeon(R) Silver 4114 CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-6.0.1 (ORCJIT, skylake)
Environment:
  JULIA_MPI_PATH = /home/eschnetter/src/spack-view


In [2]:
using Gadfly

┌ Info: Precompiling Gadfly [c91e804a-d5a3-530f-b6f0-dfbca275c004]
└ @ Base loading.jl:1273


In [ ]:
xs = [i/1000 for i in 0:1000]
plot(x=xs, y=[exp(-40*(x-0.6)^2) for x in xs], Geom.line)

## Basis functions (hat functions)

In [ ]:
"""
Basis function hat(i, N, x)
"""
function hat(i, N, x::T) where {T}
    @assert 0 <= i < N
    # calculate grid spacing
    h = 1 / T(N-1)
    # collocation points
    xim1 = h * (i-1)
    xi = h * i
    xip1 = h * (i+1)
    # case distinction (piecewise linear!)
    if x <= xim1
        return T(0)
    elseif x <= xi
        return (x - xim1) / h
    elseif x <= xip1
        return -(x - xip1) / h
    else
        return T(0)
    end
end

In [ ]:
plot(x=xs, y=[hat(4, 11, x) for x in xs], Geom.line)

## Discrete functions

In [ ]:
struct Fun{T}
    coeffs::Vector{T}
end

In [ ]:
dump(Fun{Float64}([1,2,3]))

In [ ]:
function Base.:+(f::Fun{T}, g::Fun{T}) where {T}
    Fun{T}(f.coeffs + g.coeffs)
end

In [ ]:
function Base.:-(f::Fun{T}, g::Fun{T}) where {T}
    Fun{T}(f.coeffs - g.coeffs)
end

In [ ]:
function sample(f, N)
    h = 1 / (N-1)
    coeffs = Float64[f(h*i) for i in 0:N-1]
    Fun{Float64}(coeffs)
end

In [ ]:
sample(x->x^2, 5)

In [ ]:
g(x) = exp(-40*(x-0.6)^2)

In [ ]:
g11coeffs = sample(g, 11)

In [ ]:
function evaluate(f::Fun{T}, x) where {T}
    N = length(f.coeffs)
    sum(f.coeffs[i+1] * hat(i, N, x) for i in 0:N-1)
end

In [ ]:
g11(x) = evaluate(g11coeffs, x)

In [ ]:
plot(x=xs, y=[g11(x) for x in xs], Geom.line)

In [ ]:
g21coeffs = sample(g, 21)
g21(x) = evaluate(g21coeffs, x)
plot(x=xs, y=[g21(x) for x in xs], Geom.line)

## Quadrature

In [ ]:
function quad(f::Fun{T}) where {T}
    N = length(f.coeffs)
    h = 1 / T(N-1)
    # initialize sum to zero
    s = T(0)
    for i in 0:N-1
        # boundary basis functions have half the area
        if i==0 || i==N-1
            w = T(0.5)
        else
            w = T(1)
        end
        # add up contributions from all basis functions
        s += w * h * f.coeffs[i+1]
    end
    s
end

In [ ]:
quad(g11coeffs)

In [ ]:
quad(g21coeffs)

## L2 norm

In [ ]:
function l2norm(f::Fun{T}) where {T}
    coeffs_abs2 = [abs(c)^2 for c in f.coeffs]
    sqrt(quad(Fun{T}(coeffs_abs2)))
end

In [ ]:
l2norm(g11coeffs)

In [ ]:
l2norm(g21coeffs)

## Comparing resolutions

In [ ]:
g11_21_coeffs = sample(g11, 21)

In [ ]:
plot(x=xs, y=[evaluate(g11_21_coeffs - g21coeffs, x) for x in xs], Geom.line)

In [ ]:
l2norm(g11_21_coeffs - g21coeffs)

In [ ]:
g41coeffs = sample(g, 41)
g41(x) = evaluate(g41coeffs, x)
plot(x=xs, y=[g41(x) for x in xs], Geom.line)

In [ ]:
g11_41_coeffs = sample(g11, 41)
g21_41_coeffs = sample(g21, 41)

In [ ]:
l2norm(g11_21_coeffs - g21coeffs)

In [ ]:
l2norm(g21_41_coeffs - g41coeffs)

In [ ]:
g201coeffs = sample(g, 201)
g201(x) = evaluate(g201coeffs, x)
g401coeffs = sample(g, 401)
g401(x) = evaluate(g401coeffs, x)
g201_401_coeffs = sample(g201, 401)
l2norm(g401coeffs - g201_401_coeffs)

In [ ]:
g20001coeffs = sample(g, 20001)
g20001(x) = evaluate(g20001coeffs, x)
g40001coeffs = sample(g, 40001)
g40001(x) = evaluate(g40001coeffs, x)
g20001_40001_coeffs = sample(g20001, 40001)
l2norm(g40001coeffs - g20001_40001_coeffs)